For train model we used simple baseline:
https://www.kaggle.com/code/its7171/dfl-benchmark-training

In [1]:
import cv2
import os

In [ ]:
DATASET_PATH = "../dataset"
TMP_DIR = "../tmp_video/"

In [ ]:
debug = True
if debug:
    epochs = 3
else:
    epochs = 100

err_tol = {
    'challenge': [ 0.30, 0.40, 0.50, 0.60, 0.70 ],
    'play': [ 0.15, 0.20, 0.25, 0.30, 0.35 ],
    'throwin': [ 0.15, 0.20, 0.25, 0.30, 0.35 ]
}
video_id_split = {
    'val':[
         '3c993bd2_0',
         '3c993bd2_1',
    ],
    'train':[
         '1606b0e6_0',
         '1606b0e6_1',
         '35bd9041_0',
         '35bd9041_1',
         '407c5a9e_1',
         '4ffd5986_0',
         '9a97dae4_1',
         'cfbe2e94_0',
         'cfbe2e94_1',
         'ecf251d4_0',
    ]
}
event_names = ['challenge', 'throwin', 'play']

In [ ]:
df = pd.read_csv(f"{DATASET_PATH}/input/dfl-bundesliga-data-shootout/train.csv")
additional_events = []
for arr in df.sort_values(['video_id','time','event','event_attributes']).values:
    if arr[2] in err_tol:
        tol = err_tol[arr[2]][0]/2
        additional_events.append([arr[0], arr[1]-tol, 'start_'+arr[2], arr[3]])
        additional_events.append([arr[0], arr[1]+tol, 'end_'+arr[2], arr[3]])
df = pd.concat([df, pd.DataFrame(additional_events, columns=df.columns)])
df = df[~df['event'].isin(event_names)]
df = df.sort_values(['video_id', 'time'])
df

In [ ]:
def extract_training_images(args):
        video_id, split = args
        video_path = f"{DATASET_PATH}/input/dfl-bundesliga-data-shootout/train/{video_id}.mp4"
        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        time_interval = 1/fps

        df_video = df[df.video_id == video_id]
        if debug:
            df_video = df_video.head(10)
        
        print(split, video_id, df_video.shape)

        arr = df_video[['time','event']].values
        for idx in range(len(arr)-1):
            crr_time = arr[idx,0]
            nxt_time = arr[idx+1,0]
            crr_event = arr[idx,1]

            crr_event = crr_event
            if crr_event == 'start':
                crr_status = 'background'
            elif crr_event == 'end':
                continue
            else:
                start_or_end, crr_status = crr_event.split('_', 1)
                if start_or_end == 'end':
                    crr_status = 'background'

            result_dir = f"{TMP_DIR}/split_images/{split}/{crr_status}"
            if not os.path.exists(result_dir):
                os.makedirs(result_dir, exist_ok=True)
            
            this_time = crr_time
            while this_time < nxt_time:
                frame_num = int(this_time*fps)

                cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
                ret, frame = cap.read()
                out_file = f'{result_dir}/{video_id}_{frame_num:06}.jpg'
                cv2.imwrite(out_file, frame)

                if crr_status == 'background':
                    this_time += time_interval*10
                else:
                    this_time += time_interval

In [ ]:
!rm -rf ../work/split_images/

In [ ]:
for split in video_id_split:
    video_ids = video_id_split[split]
    for video_id in video_ids:            
        extract_training_images([video_id, split])

In [ ]:
if not os.path.exists(f"{TMP_DIR}/train_model"):
    os.makedirs(f"{TMP_DIR}/train_model", exist_ok=True)

In [ ]:
!python ./train.py $TMP_DIR/split_images \
    -b 10 \
    --amp \
    --epochs $epochs \
    --pretrained \
    --num-classes 4 \
    --model tf_efficientnet_b5_ap \
    --output $TMP_DIR/train_model